In [1]:
import os
import cv2

import numpy as np
import matplotlib.pyplot as plt

from modelLoader import segmentationModel

PATH_imagenes = 'C:/Users/abcar/Desktop/casi TODO/ESCUELA/INTERNADO/Resultados/U-NET/fundus/Imagenes/'
PATH_mascaras = 'C:/Users/abcar/Desktop/casi TODO/ESCUELA/INTERNADO/Resultados/U-NET/fundus/Mascaras/'
PATH_modelo = 'C:/Users/abcar/Desktop/casi TODO/ESCUELA/INTERNADO/Resultados/U-NET/Modelos/fundus.tflite'

listimagenes = os.listdir(PATH_imagenes)
listmascaras = os.listdir(PATH_mascaras)

modelo = segmentationModel(PATH_modelo)
dim = modelo.getDims()

print("Imagenes encontradas {}".format(len(listimagenes)))
print("Dimensiones esperadas {}".format(dim))

Imagenes encontradas 1020
Dimensiones esperadas (256, 256)


In [2]:
def dice_coef(true, pred):

    inter = np.sum(true*pred)
    total = np.sum(pred) + np.sum(true)

    return np.round((2*inter)/total, 3)

def leerImagenes(PATH : str, dim, im = 2000):

    imagenes = sorted(os.listdir(PATH)) # Listamos las imágenes en el path.
    im = [cv2.cvtColor(cv2.resize(cv2.imread(PATH + imagenes[i]), dim), cv2.COLOR_BGR2GRAY) for i in range(0, im)]

    return im

imagenes = leerImagenes(PATH_imagenes, dim, len(listimagenes))
imagenes = [i/np.max(i) for i in imagenes]

true = leerImagenes(PATH_mascaras, dim, len(listimagenes))
truenorm = [i/np.max(i) if np.max(i) != 0 else i for i in true]

In [15]:
truenorm_bin = [cv2.threshold(i, 0.4, 1, cv2.THRESH_BINARY)[1] for i in truenorm]

In [30]:
import math

conteo, score = 0, []
im = []

no_proces = []

for i in range(0, len(imagenes)):

    pred = modelo.predict(imagenes[i], contornos = False)
    s = dice_coef(truenorm_bin[i], pred)

    if s == 0 or math.isnan(s):

        no_proces.append(i)
        score.append(0)

    else:

        conteo += 1

        im.append(i) # Guardamos la imagen.
        score.append(s)

scoreaux = np.array(score)

print(conteo)
print(np.average(score))
print(np.std(score))

1017
0.9134029411764706
0.07980313640192829


In [31]:
print(len(score))

1020


In [32]:
iou_score, conteo = [], 0

for i in im:

    intersection = np.logical_and(truenorm[i], modelo.predict(imagenes[i], contornos = False))
    union = np.logical_or(truenorm[i], modelo.predict(imagenes[i], contornos = False))
    iou = np.sum(intersection)/np.sum(union)

    if iou == 0 or math.isnan(iou):

        iou_score.append(0)

    else:

        conteo += 1
        iou_score.append(iou)

print(conteo)
print(np.average(iou_score))
print(np.std(iou_score))

1017
0.8501372527166907
0.08757887677110436


In [30]:
from sklearn.metrics import confusion_matrix

pixel_accuracy = []

for i in im:

    predi = np.array(modelo.predict(imagenes[i], contornos = False))
    cm = confusion_matrix(np.array(truenorm[i], dtype = np.uint8).flatten(), predi.flatten())
    proper = np.sum(np.diagonal(cm))
    pix_acc = proper/(256**2)

    pixel_accuracy.append(pix_acc)

print(np.average(pixel_accuracy))
print(np.std(pixel_accuracy))

0.9581531897446557
0.03888687594379214
